In [12]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
from sklearn.decomposition import PCA
import sys
import pandas as pd
import seaborn as sns
import xarray as xr

sys.path.append('/Users/sr2/opt/anaconda3/envs/nipr_env/')
sys.path.append('/home/ryuho/anaconda3/envs/smrai_env')
from esn_dts_openloop import ESN, Tikhonov

In [13]:
#REPPU otuputs
#path = r'/Users/sr2/OneDrive - University College London/PhD/Experiences/Postdocs/JSPS/Research/Data/pbig5min.dat'
path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/xbig5min.dat'

#read the REPPU data
with open (path) as f:
    rectype = np.dtype(np.float32)
    reppu_x = np.fromfile(f, rectype) #size = 109,900,800

In [14]:
#REPPU otuputs
#path = r'/Users/sr2/OneDrive - University College London/PhD/Experiences/Postdocs/JSPS/Research/Data/pbig5min.dat'
path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/ybig5min.dat'

#read the REPPU data
with open (path) as f:
    rectype = np.dtype(np.float32)
    reppu_y = np.fromfile(f, rectype) #size = 109,900,800

reppu_y.shape

(109900800,)

In [15]:
matrix = np.array([[reppu_x, reppu_y], [-reppu_x, reppu_y]])
matrix.shape

(2, 2, 109900800)

In [16]:
hall_xy = matrix[0,1]
hall_neg_xy = matrix[1,0]

In [17]:
hall = np.trace(matrix)
hall.shape

(109900800,)

In [18]:
mhd_data = pd.read_csv('mhd_dates.csv')
expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
        for start, end in zip(mhd_data['start'], mhd_data['end'])])

In [19]:
# Reshape the array to have dimensions (days, time (5-min intervals), lat, lon)
#288 will become 1440 at 1-min resolution in SMRAI v3.0
data_reshaped = hall.reshape(len(expanded_dt), 288, 30, 80) 
0
# Define coordinates
time = np.arange(288)
lat = np.linspace(53.1, 89.7, 30) #30 intervals between 53.1° to 89.7°
lon = np.linspace(1.6, 357.6, 80) #80 intervals between 1.6° to 357.6°

# Create 'dt' variable combining dates and five-minute intervals
dt = []
for day in expanded_dt:
    for t in time:
        dt.append(day + pd.Timedelta(minutes=t*5))
dt = np.array(dt) #convert from list to numpy array

# Create xarray Dataset
ds = xr.Dataset({'conductivity': (['dt', 'lat', 'lon'], data_reshaped.reshape(-1, 30, 80))},
                coords={'dt': dt, 'lat': lat, 'lon': lon})

# Add potential and units
ds['conductivity'].attrs['units'] = 'S/m'
ds

<xarray.Dataset>
Dimensions:       (dt: 45792, lat: 30, lon: 80)
Coordinates:
  * dt            (dt) datetime64[ns] 2021-05-10 ... 2022-07-31T23:55:00
  * lat           (lat) float64 53.1 54.36 55.62 56.89 ... 87.18 88.44 89.7
  * lon           (lon) float64 1.6 6.106 10.61 15.12 ... 348.6 353.1 357.6
Data variables:
    conductivity  (dt, lat, lon) float32 3.468 3.468 3.468 ... 9.777 9.773 9.769

In [20]:
x = 1.84
y = 1.62
w = 1.51

In [21]:
xx = x * x #top left
xy = x * y #top right
_xy = -x * -y #bottom left
yy = y * y #bottom right

In [22]:
#print a;ll and round to nearest 2 dp
print('xx:', round(xx, 2))
print('xy:', round(xy, 2))
print('_xy:', round(_xy, 2))
print('yy:', round(yy, 2))

xx: 3.39
xy: 2.98
_xy: 2.98
yy: 2.62
